In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# hyper parameters
EPOCH = 2
BATCH_SIZE = 50
LR = 0.001
DOWNLOAD_MNIST = False

In [3]:
train_data = torchvision.datasets.MNIST(
    root='./mnist',
    train=True,
    transform=torchvision.transforms.ToTensor(),  # value : (0,1)
    download=DOWNLOAD_MNIST
)

In [4]:
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
# convert test data into Variable, pick 2000 samples to speed up testing
# add cuda()
test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)
test_x = Variable(torch.unsqueeze(test_data.test_data, dim=1), volatile=True).type(torch.FloatTensor)[:2000].cuda()/255.   # shape from (2000, 28, 28) to (2000, 1, 28, 28), value in range(0,1)
test_y = test_data.test_labels[:2000].cuda()


In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (1, 28, 28)
            nn.Conv2d(
                in_channels=1,              # input height
                out_channels=16,            # n_filters
                kernel_size=5,              # filter size
                stride=1,                   # filter movement/step
                padding=2,                  # if want same width and length of this image after con2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (16, 28, 28)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(kernel_size=2),    # choose max value in 2x2 area, output shape (16, 14, 14)
        )
        self.conv2 = nn.Sequential(         # input shape (16, 14, 14)
            nn.Conv2d(16, 32, 5, 1, 2),     # output shape (32, 14, 14)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(2),                # output shape (32, 7, 7)
        )
        self.out = nn.Linear(32 * 7 * 7, 10)   # fully connected layer, output 10 classes

    def forward(self, x):                   # in forward, needs to consider batch_size dim.
        x = self.conv1(x)
        x = self.conv2(x)                   # -> (batch_size, 32, 7, 7)
        x = x.view(x.size(0), -1)           # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output

In [8]:
cnn = CNN()
cnn.cuda()  # Moves all model parameters and buffers to the GPU.
print(cnn)

CNN (
  (conv1): Sequential (
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU ()
    (2): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (conv2): Sequential (
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU ()
    (2): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (out): Linear (1568 -> 10)
)


In [9]:
# Define the optimizer and loss function
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()   # the target label is  not one-hotted

In [10]:
# training and testing
for epoch in range(EPOCH):
    for step, (x,y) in enumerate(train_loader):  # gives batch data, normalize x 
        b_x = Variable(x).cuda()   # batch x, add cuda()
        b_y = Variable(y).cuda()   # batch y, add cuda()
        
        output = cnn(b_x)   # cnn output
        loss = loss_func(output, b_y)  # cross_entropy loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % 50 == 0:
            test_output = cnn(test_x)
            pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze()
            accuracy = sum(pred_y == test_y) / float(test_y.size(0))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data[0], '| test accuracy: %.4f' % accuracy)

Epoch:  0 | train loss: 2.3174 | test accuracy: 0.1980
Epoch:  0 | train loss: 0.6783 | test accuracy: 0.8075
Epoch:  0 | train loss: 0.2944 | test accuracy: 0.8855
Epoch:  0 | train loss: 0.2428 | test accuracy: 0.9130
Epoch:  0 | train loss: 0.1198 | test accuracy: 0.9340
Epoch:  0 | train loss: 0.1148 | test accuracy: 0.9280
Epoch:  0 | train loss: 0.0770 | test accuracy: 0.9425
Epoch:  0 | train loss: 0.1487 | test accuracy: 0.9505
Epoch:  0 | train loss: 0.0279 | test accuracy: 0.9635
Epoch:  0 | train loss: 0.2090 | test accuracy: 0.9625
Epoch:  0 | train loss: 0.0783 | test accuracy: 0.9690
Epoch:  0 | train loss: 0.0220 | test accuracy: 0.9705
Epoch:  0 | train loss: 0.0636 | test accuracy: 0.9710
Epoch:  0 | train loss: 0.0858 | test accuracy: 0.9735
Epoch:  0 | train loss: 0.0721 | test accuracy: 0.9710
Epoch:  0 | train loss: 0.0312 | test accuracy: 0.9670
Epoch:  0 | train loss: 0.0188 | test accuracy: 0.9780
Epoch:  0 | train loss: 0.1066 | test accuracy: 0.9750
Epoch:  0 